In [1]:
pip show openai


Name: openai
Version: 0.28.0
Summary: Python client library for the OpenAI API
Home-page: https://github.com/openai/openai-python
Author: OpenAI
Author-email: support@openai.com
License: 
Location: p:\generative_ai\venv\lib\site-packages
Requires: aiohttp, requests, tqdm
Required-by: 
Note: you may need to restart the kernel to use updated packages.


In [12]:
import pandas as pd
import openai
openai.api_base = "https://openai.vocareum.com/v1"
openai.api_key = "voc-605570984126677350407866e29880939962.25846762"
# Load the dataset
df = pd.read_csv('nyc_food_scrap_drop_off_sites.csv')
df.head()


,Borough,NTAName,SiteName,SiteAddr,Hosted_By,Open_Month,Day_Hours,Notes,Website,BoroCD,CouncilDis,ct2010,BBL,BIN,Latitude,Longitude,PolicePrec,Object ID,Location Point
0,Manhattan,Inwood,SW West 212th Street & 10th Avenue,NaN,Department of Sanitation,Year Round,24/7,Download the app to access bins. Accepts all f...,www.nyc.gov/smartcomposting,112,10,NaN,NaN,NaN,NaN,NaN,34,6914,NaN
1,Brooklyn,Park Slope,Old Stone House Brooklyn,"336 3rd St, Brooklyn, NY 11215",Old Stone House Brooklyn,Year Round,24/7 (Start Time: 24/7 - End Time: 24/7),NaN,NaN,306,39,NaN,NaN,NaN,40.672712,-73.984731,78,6777,POINT (-73.984731 40.6727118)
2,Bronx,Norwood,NE Hull Avenue & East 207th Street,NaN,Department of Sanitation,Year Round,24/7,Download the app to access bins. Accepts all f...,www.nyc.gov/smartcomposting,207,11,NaN,NaN,NaN,NaN,NaN,52,7115,NaN
3,Manhattan,East Harlem (South),SE East 112th Street & Park Avenue,NaN,Department of Sanitation,Year Round,24/7,Download the app to access bins. Accepts all f...,www.nyc.gov/smartcomposting,111,8,NaN,NaN,NaN,NaN,NaN,23,6819,NaN
4,Queens,Astoria (Central),Astoria Pug: Broadway,"32-11 31st St, Queens, NY 11106",Astoria Pug,Year Round,Saturdays (Start Time: 9:00 AM - End Time: 3:...,"Not accepted: meat, bones, or dairy",https://www.instagram.com/astoriapug/?hl=en,401,22,NaN,NaN,NaN,40.761472,-73.925611,114,7046,POINT (-73.925611 40.761472)


In [13]:
# Remove unnecessary columns
df_cleaned = df[['Borough', 'NTAName',  'Notes']]

# Display the cleaned dataframe
df_cleaned.head()

,Borough,NTAName,Notes
0,Manhattan,Inwood,Download the app to access bins. Accepts all f...
1,Brooklyn,Park Slope,NaN
2,Bronx,Norwood,Download the app to access bins. Accepts all f...
3,Manhattan,East Harlem (South),Download the app to access bins. Accepts all f...
4,Queens,Astoria (Central),"Not accepted: meat, bones, or dairy"


In [21]:
# Function to handle chatbot queries
def chatbot_query(prompt):
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": prompt}
        ],
        max_tokens=150,
        n=1,
        stop=None,
        temperature=0.7,
    )
    # print(response)
    return response.choices[0].message.content.strip()


In [22]:
# Example user interaction
user_input = "Where can I drop off food scraps in Brooklyn?"
bot_response = chatbot_query(user_input)
print(bot_response)

You can drop off food scraps for composting at various locations in Brooklyn. Some options include community compost drop-off sites, farmers markets, and select greenmarkets. You can also check with your local community garden or composting organization for drop-off locations near you. Additionally, some residential buildings and local businesses offer composting services. It's always a good idea to call ahead or check online for the most up-to-date information on drop-off locations in Brooklyn.


In [24]:
def search_drop_off_location(borough):
    # Filter based on borough
    results = df_cleaned[df_cleaned['Borough'].str.contains(borough, case=False, na=False)]
    return results

user_borough = "Brooklyn"
locations = search_drop_off_location(user_borough)

# Display top 5 results for simplicity
print(locations.head())


    Borough                    NTAName  \
1  Brooklyn                 Park Slope   
5  Brooklyn      East New York (North)   
6  Brooklyn  Bedford-Stuyvesant (East)   
7  Brooklyn      Crown Heights (North)   
8  Brooklyn            Bushwick (West)   

                                               Notes  
1                                                NaN  
5                Not accepted: meat, bones, or dairy  
6  Download the app to access bins. Accepts all f...  
7  Download the app to access bins. Accepts all f...  
8  Download the app to access bins. Accepts all f...  


In [28]:
def chatbot_with_data(query, borough):
    # Get AI-generated response
    ai_response = chatbot_query(query)
    
    # Filter data based on borough
    borough_data = search_drop_off_location(borough)

    print(borough_data)
    # Format the response
    response = f"{ai_response}\nHere are some drop-off locations in {borough}:\n{borough_data['NTAName'].head().to_string(index=False)}"
    return response

# Example interaction
user_query = "Where can I drop off food scraps in Brooklyn?"
print(chatbot_with_data(user_query, "Brooklyn"))


      Borough                    NTAName  \
1    Brooklyn                 Park Slope   
5    Brooklyn      East New York (North)   
6    Brooklyn  Bedford-Stuyvesant (East)   
7    Brooklyn      Crown Heights (North)   
8    Brooklyn            Bushwick (West)   
..        ...                        ...   
473  Brooklyn            Bushwick (East)   
474  Brooklyn            Bushwick (East)   
476  Brooklyn                 Ocean Hill   
477  Brooklyn            Bushwick (East)   
478  Brooklyn      East New York (North)   

                                                 Notes  
1                                                  NaN  
5                  Not accepted: meat, bones, or dairy  
6    Download the app to access bins. Accepts all f...  
7    Download the app to access bins. Accepts all f...  
8    Download the app to access bins. Accepts all f...  
..                                                 ...  
473  Download the app to access bins. Accepts all f...  
474  Download t